# Abordagem Inicial 

## Objetivo 

### Propor método para:
* encontrar representações comuns de diferentes modalidades sensoriais
* aumentar a quantidade de dados utilizados para redução de dimensionalidade do sinal
* melhorar a sensibilidade de análises MVPA posteriores 

### Inicialmente codificar cada instante de tempo (sem usar janelas no tempo)
### Decodificar o momento que estava na mesma e na outra modalidade a partir do estado reduzido
* Comparar com decodificação usando: (1) Sinais originais, (2) PCA geral, (3) PCA em cada ponto, (4) Auto-encoder
### Ver sobre use da auto-encoders e VAEs
### Checar sobre predição da reprodução do intervalo temporal


In [1]:
from myMNE import makeMNE

In [2]:
from os import listdir
from os.path import isfile, join
mypath = 'aud'

onlyfiles = [mypath+"/"+f for f in listdir(mypath) if isfile(join(mypath, f))]
auds = list(map(makeMNE,onlyfiles))

240 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
0 bad epochs dropped
240 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
0 bad epochs dropped
240 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
0 bad epochs dropped
240 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
0 bad epochs dropped
240 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
0 bad epochs dropped
240 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
0 bad epochs dropped
240 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
0 bad epochs dropped
240 matching events found
A

### Passo 1: treinar com dados completos capturados do experimento
* Entrada e saída como o mesmo sinal (um número para cada eletrodo)
* Usar uma rede fully connected e auto-encoder padrão
    * 64(E) - 32 - 16 - 8 - 16 - 32 - 64(S)
    * Fazer testes verificando o erro de reconstrução
* Uma rede por voluntário e por modalidade
* Pouca quantidade de dados -> incluir regularização, como dropout, para evitar overfitting


In [21]:
data = [aud.get_data().T for aud in auds]

In [22]:
import numpy as np
merge_data = np.concatenate(data)

In [28]:
merge_data =  merge_data.reshape((10780, 240, 64))

In [29]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import losses

In [31]:
fun_loss = losses.mean_absolute_error

value_encoding_dim = 8
original_trial =  Input(shape=(240,1))
enconded = Dense(64, activation='relu')(original_trial)
enconded = Dense(32, activation='relu')(enconded)
enconded = Dense(16, activation='relu')(enconded)


enconded = Dense(value_encoding_dim, activation='relu')(enconded)
deconded = Dense(16, activation='relu', use_bias=False)(enconded)
deconded = Dense(32, activation='relu', use_bias=False)(deconded)
deconded = Dense(64, activation='relu', use_bias=False)(deconded)

encoder = Model(original_trial, enconded, name='encoder')
autoencoder = Model(original_trial, deconded, name='autoenconder')    


autoencoder.compile(optimizer='adam', loss=fun_loss,metrics = ['accuracy'])

In [37]:
canal_1 = merge_data[:,:,63]
X_exemplo = canal_1.reshape(10780, 240,1)

In [ ]:
epochs = 5000
batch_size = 500

history = autoencoder.fit(X_exemplo,X_exemplo, epochs=epochs, batch_size=batch_size,shuffle=True,validation_data=(X_exemplo,X_exemplo))

Train on 10780 samples, validate on 10780 samples
Epoch 1/5000
10780/10780 [==============================] - 0s 37us/sample - loss: 7.7646 - accuracy: 0.5012 - val_loss: 7.7631 - val_accuracy: 0.5012
Epoch 2/5000
10780/10780 [==============================] - 0s 42us/sample - loss: 7.7643 - accuracy: 0.5012 - val_loss: 7.7645 - val_accuracy: 0.5012
Epoch 3/5000
10780/10780 [==============================] - 0s 41us/sample - loss: 7.7660 - accuracy: 0.5012 - val_loss: 7.7693 - val_accuracy: 0.5012
Epoch 4/5000
10780/10780 [==============================] - 0s 43us/sample - loss: 7.7640 - accuracy: 0.5012 - val_loss: 7.7638 - val_accuracy: 0.5012
Epoch 5/5000
10780/10780 [==============================] - 0s 41us/sample - loss: 7.7631 - accuracy: 0.5012 - val_loss: 7.7624 - val_accuracy: 0.5012
Epoch 6/5000
10780/10780 [==============================] - 0s 43us/sample - loss: 7.7631 - accuracy: 0.5012 - val_loss: 7.7621 - val_accuracy: 0.5012
Epoch 7/5000
10780/10780 [==================

10780/10780 [==============================] - 0s 31us/sample - loss: 7.7689 - accuracy: 0.5012 - val_loss: 7.7630 - val_accuracy: 0.5012
Epoch 55/5000
10780/10780 [==============================] - 0s 37us/sample - loss: 7.7662 - accuracy: 0.5012 - val_loss: 7.7685 - val_accuracy: 0.5012
Epoch 56/5000
10780/10780 [==============================] - 0s 36us/sample - loss: 7.7656 - accuracy: 0.5940 - val_loss: 7.7636 - val_accuracy: 0.5012
Epoch 57/5000
10780/10780 [==============================] - 0s 36us/sample - loss: 7.7634 - accuracy: 0.5012 - val_loss: 7.7636 - val_accuracy: 0.5012
Epoch 58/5000
10780/10780 [==============================] - 0s 36us/sample - loss: 7.7634 - accuracy: 0.5012 - val_loss: 7.7636 - val_accuracy: 0.5012
Epoch 59/5000
10780/10780 [==============================] - 0s 37us/sample - loss: 7.7643 - accuracy: 0.5012 - val_loss: 7.7630 - val_accuracy: 0.5012
Epoch 60/5000
10780/10780 [==============================] - 0s 40us/sample - loss: 7.7660 - accuracy:

Epoch 108/5000
10780/10780 [==============================] - 0s 37us/sample - loss: 7.7646 - accuracy: 0.5472 - val_loss: 7.7652 - val_accuracy: 0.5012
Epoch 109/5000
10780/10780 [==============================] - 0s 40us/sample - loss: 7.7676 - accuracy: 0.5012 - val_loss: 7.7661 - val_accuracy: 0.5012
Epoch 110/5000
10780/10780 [==============================] - 0s 39us/sample - loss: 7.7698 - accuracy: 0.5930 - val_loss: 7.7681 - val_accuracy: 0.5012
Epoch 111/5000
10780/10780 [==============================] - 0s 39us/sample - loss: 7.7698 - accuracy: 0.5012 - val_loss: 7.7725 - val_accuracy: 0.5011
Epoch 112/5000
10780/10780 [==============================] - 0s 38us/sample - loss: 7.7685 - accuracy: 0.5012 - val_loss: 7.7718 - val_accuracy: 0.5011
Epoch 113/5000
10780/10780 [==============================] - 1s 52us/sample - loss: 7.7678 - accuracy: 0.5012 - val_loss: 7.7663 - val_accuracy: 0.5012
Epoch 114/5000
10780/10780 [==============================] - 0s 37us/sample - los

10780/10780 [==============================] - 0s 34us/sample - loss: 7.7638 - accuracy: 0.5936 - val_loss: 7.7650 - val_accuracy: 0.5013
Epoch 162/5000
10780/10780 [==============================] - 0s 38us/sample - loss: 7.7632 - accuracy: 0.5012 - val_loss: 7.7621 - val_accuracy: 0.5011
Epoch 163/5000
10780/10780 [==============================] - 0s 38us/sample - loss: 7.7640 - accuracy: 0.5012 - val_loss: 7.7638 - val_accuracy: 0.5012
Epoch 164/5000
10780/10780 [==============================] - 0s 37us/sample - loss: 7.7632 - accuracy: 0.5012 - val_loss: 7.7629 - val_accuracy: 0.5012
Epoch 165/5000
10780/10780 [==============================] - 0s 37us/sample - loss: 7.7631 - accuracy: 0.5012 - val_loss: 7.7625 - val_accuracy: 0.5012
Epoch 166/5000
10780/10780 [==============================] - 0s 37us/sample - loss: 7.7699 - accuracy: 0.5012 - val_loss: 7.7655 - val_accuracy: 0.5012
Epoch 167/5000
10780/10780 [==============================] - 0s 39us/sample - loss: 7.7634 - acc

10780/10780 [==============================] - 0s 40us/sample - loss: 7.7634 - accuracy: 0.5012 - val_loss: 7.7623 - val_accuracy: 0.5012
Epoch 215/5000
10780/10780 [==============================] - 0s 40us/sample - loss: 7.7680 - accuracy: 0.5012 - val_loss: 7.7653 - val_accuracy: 0.5012
Epoch 216/5000
10780/10780 [==============================] - 0s 40us/sample - loss: 7.7647 - accuracy: 0.5012 - val_loss: 7.7639 - val_accuracy: 0.5012
Epoch 217/5000
10780/10780 [==============================] - 0s 40us/sample - loss: 7.7634 - accuracy: 0.5012 - val_loss: 7.7637 - val_accuracy: 0.5012
Epoch 218/5000
10780/10780 [==============================] - 0s 46us/sample - loss: 7.7627 - accuracy: 0.5012 - val_loss: 7.7636 - val_accuracy: 0.5012
Epoch 219/5000
10780/10780 [==============================] - 1s 47us/sample - loss: 7.7641 - accuracy: 0.5139 - val_loss: 7.7678 - val_accuracy: 1.0000
Epoch 220/5000
10780/10780 [==============================] - 1s 47us/sample - loss: 7.7659 - acc

...

10780/10780 [==============================] - 0s 36us/sample - loss: 7.7619 - accuracy: 0.5012 - val_loss: 7.7625 - val_accuracy: 0.5012
Epoch 1063/5000
10780/10780 [==============================] - 0s 30us/sample - loss: 7.7624 - accuracy: 0.5012 - val_loss: 7.7611 - val_accuracy: 0.5012
Epoch 1064/5000
10780/10780 [==============================] - 0s 37us/sample - loss: 7.7624 - accuracy: 0.5012 - val_loss: 7.7621 - val_accuracy: 0.5011
Epoch 1065/5000
10780/10780 [==============================] - 0s 35us/sample - loss: 7.7629 - accuracy: 0.5012 - val_loss: 7.7616 - val_accuracy: 0.5012
Epoch 1066/5000
10780/10780 [==============================] - 0s 30us/sample - loss: 7.7630 - accuracy: 0.5012 - val_loss: 7.7621 - val_accuracy: 0.5012
Epoch 1067/5000
10780/10780 [==============================] - 0s 40us/sample - loss: 7.7643 - accuracy: 0.5012 - val_loss: 7.7607 - val_accuracy: 0.5012
Epoch 1068/5000
10780/10780 [==============================] - 0s 38us/sample - loss: 7.7620

10780/10780 [==============================] - 0s 46us/sample - loss: 7.7645 - accuracy: 0.5012 - val_loss: 7.7620 - val_accuracy: 0.5012
Epoch 1116/5000
10780/10780 [==============================] - 0s 36us/sample - loss: 7.7621 - accuracy: 0.5012 - val_loss: 7.7627 - val_accuracy: 0.5012
Epoch 1117/5000
10780/10780 [==============================] - 0s 34us/sample - loss: 7.7619 - accuracy: 0.5012 - val_loss: 7.7623 - val_accuracy: 0.5012
Epoch 1118/5000
10780/10780 [==============================] - 0s 44us/sample - loss: 7.7618 - accuracy: 0.5012 - val_loss: 7.7620 - val_accuracy: 0.5012
Epoch 1119/5000
10780/10780 [==============================] - 0s 40us/sample - loss: 7.7627 - accuracy: 0.5012 - val_loss: 7.7641 - val_accuracy: 0.5012
Epoch 1120/5000
10780/10780 [==============================] - 0s 32us/sample - loss: 7.7628 - accuracy: 0.5012 - val_loss: 7.7621 - val_accuracy: 0.5012
Epoch 1121/5000
10780/10780 [==============================] - 0s 32us/sample - loss: 7.7660

10780/10780 [==============================] - 0s 32us/sample - loss: 7.7620 - accuracy: 0.5012 - val_loss: 7.7615 - val_accuracy: 0.5011
Epoch 1169/5000
10780/10780 [==============================] - 0s 39us/sample - loss: 7.7620 - accuracy: 0.5012 - val_loss: 7.7631 - val_accuracy: 0.5012
Epoch 1170/5000
10780/10780 [==============================] - 0s 34us/sample - loss: 7.7625 - accuracy: 0.5012 - val_loss: 7.7625 - val_accuracy: 0.5012
Epoch 1171/5000
10780/10780 [==============================] - 0s 34us/sample - loss: 7.7631 - accuracy: 0.5012 - val_loss: 7.7638 - val_accuracy: 0.5012
Epoch 1172/5000
10780/10780 [==============================] - 0s 39us/sample - loss: 7.7629 - accuracy: 0.5012 - val_loss: 7.7634 - val_accuracy: 0.5012
Epoch 1173/5000
10780/10780 [==============================] - 0s 30us/sample - loss: 7.7630 - accuracy: 0.5373 - val_loss: 7.7646 - val_accuracy: 1.0000
Epoch 1174/5000
10780/10780 [==============================] - 0s 36us/sample - loss: 7.7625

10780/10780 [==============================] - 0s 38us/sample - loss: 7.7630 - accuracy: 0.5012 - val_loss: 7.7647 - val_accuracy: 0.5011
Epoch 1222/5000
10780/10780 [==============================] - 0s 37us/sample - loss: 7.7634 - accuracy: 0.5477 - val_loss: 7.7613 - val_accuracy: 0.5012
Epoch 1223/5000
10780/10780 [==============================] - 0s 32us/sample - loss: 7.7619 - accuracy: 0.5609 - val_loss: 7.7629 - val_accuracy: 1.0000
Epoch 1224/5000
10780/10780 [==============================] - 0s 36us/sample - loss: 7.7622 - accuracy: 0.5703 - val_loss: 7.7620 - val_accuracy: 0.5011
Epoch 1225/5000
10780/10780 [==============================] - 0s 36us/sample - loss: 7.7618 - accuracy: 0.5012 - val_loss: 7.7649 - val_accuracy: 0.5012
Epoch 1226/5000
10780/10780 [==============================] - 0s 32us/sample - loss: 7.7628 - accuracy: 0.5012 - val_loss: 7.7627 - val_accuracy: 0.5012
Epoch 1227/5000
10780/10780 [==============================] - 0s 32us/sample - loss: 7.7628

10780/10780 [==============================] - 0s 41us/sample - loss: 7.7648 - accuracy: 0.5012 - val_loss: 7.7614 - val_accuracy: 0.5012
Epoch 1275/5000
10780/10780 [==============================] - 0s 41us/sample - loss: 7.7624 - accuracy: 0.5012 - val_loss: 7.7625 - val_accuracy: 0.5012
Epoch 1276/5000
10780/10780 [==============================] - 0s 41us/sample - loss: 7.7635 - accuracy: 0.5012 - val_loss: 7.7632 - val_accuracy: 0.5012
Epoch 1277/5000
10780/10780 [==============================] - 0s 42us/sample - loss: 7.7627 - accuracy: 0.5012 - val_loss: 7.7655 - val_accuracy: 0.5012
Epoch 1278/5000
10780/10780 [==============================] - 0s 41us/sample - loss: 7.7686 - accuracy: 0.5012 - val_loss: 7.7658 - val_accuracy: 0.5012
Epoch 1279/5000
10780/10780 [==============================] - 0s 41us/sample - loss: 7.7649 - accuracy: 0.5012 - val_loss: 7.7629 - val_accuracy: 0.5012
Epoch 1280/5000
10780/10780 [==============================] - 0s 32us/sample - loss: 7.7628

In [40]:
import matplotlib.pyplot as plt

print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()




### Passo 2: gerar uma representação comum entre ambas as modalidades
* Treinar com a mesma rede para visual e auditivo
* Objetivo é verificar se gera uma melhor representação comum entre as modalidades
* Comparar o desempenho com a rede treinada em apenas uma modalidade
* Treinar redes separadas para visual e auditivo, compartilhando a parte da codificação
* Treinamento com exemplos de ambas as modalidades
* Pode ser mais interessante, pois enfatiza uma codificação comum, mas permite diferenciações na decodificação

### Passo 3: obter representações comuns entre sujeitos
* Treinar a mesma rede para todos os participantes 
* Objetivo é gerar as codificações a partir de grandes quantidades de dados
* Abordagem 1: Fazer para a rede uma rede codificação/decodificação comum a todos
    * Depois pode ser afinada para cada participante individualmente
* Abordagem 2: Usar apenas a parte de codificação em comum
    * Enfatiza uma codificação comum, mas permite diferenciações na decodificação
* Neste caso o classificador poderia ser comum também?

Passo 4: capturar informação temporal (opcional)
Repetir os passos anteriores, mas usando 5 valores para cada eletrodo (1 a cada 25 ms)
Objetivo é verificar se mais detalhes temporais melhoram a representação do tempo
Inicialmente utilizar uma arquitetura fully connected 
320(E) - 160 - 80 - 40 - 80 - 160 - 320(S)
Outra possibilidade é colocar alguma estrutura
Transformação é inicialmente realizada sobre a série temporal de cada eletrodo
Poderia também juntar sinais de eletrodos vizinhos
Isso permitiria diminuir o número de conexões